In [1]:
import rdflib
from rdflib import Graph
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as plt

import requests
import json

In [38]:
import pandas as pd 
import networkx as nx
import nx_altair as nx
import altair as alt

In [21]:
ask_query = """ASK {
      wd:Q34201 p:P31 ?statement0.
      ?statement0 (ps:P31/(wdt:P279*)) wd:Q215627.
 }"""


endpoint_url = "https://query.wikidata.org/sparql"
headers = {'User-Agent': 'MyBot'}
payload = {
        'query': ask_query,
        'format': 'json'
    }
r = requests.get(endpoint_url, params=payload, headers=headers)

In [24]:
r.status_code #if status_code is 400 then error in formatting the request

200

In [26]:
r.text

'{\n  "head" : { },\n  "boolean" : true\n}'

In [31]:
r.json()['boolean']

True

# defining the classifyer

In [35]:
def is_person(wiki_code, with_wd=False):
    """from the wikipedia code returns whether the entity is a person or not"""
    if with_wd:
        code = wiki_code
    else : code = "wd:"+wiki_code
    ask_query = "ASK {"+ code +" p:P31 ?statement0. ?statement0 (ps:P31/(wdt:P279*)) wd:Q215627.}"
    endpoint_url = "https://query.wikidata.org/sparql"
    headers = {'User-Agent': 'MyBot'}
    payload = {
            'query': ask_query,
            'format': 'json'
        }
    r = requests.get(endpoint_url, params=payload, headers=headers)
    return r.json()['boolean']

In [36]:
is_person("Q5383") #test sur david bowie

True

# identifying the interesting elements
has been done by finding the right terms
- person wd:Q215627
- geographic entity wd:Q27096213
- string wd:Q184754
- temporal entity wd:Q26907166
- group of humans wd:Q16334295
- wikipedia related
- other

## classifying the types of people

### putting away the wikipedia and other links

In [42]:
def is_wikidata(iri):
    caract = "https://www.wikidata.org/"
    taille = len(caract)
    return iri[:taille] == caract

In [44]:
is_wikidata("https://www.wikipedia.org/wiki/Q16334295")

False

# Testing on a smaller graph

In [39]:
PATH = "collecting_data_with_SPARQL/graph_to_queer_people_dist1.csv"
df = pd.read_csv(PATH)
df2 = df[:100]

### testing with only the human

In [40]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   subject    100 non-null    object 
 1   predicate  100 non-null    object 
 2   object     100 non-null    object 
 3   context    0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 3.2+ KB
